In [1]:
pip install pennylane torch scikit-learn

   ---------------------------------------- 0.0/203.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/203.1 MB ? eta -:--:--
   ---------------------------------------- 0.3/203.1 MB ? eta -:--:--
   ---------------------------------------- 0.3/203.1 MB ? eta -:--:--
   ---------------------------------------- 0.5/203.1 MB 645.7 kB/s eta 0:05:14
   ---------------------------------------- 0.5/203.1 MB 645.7 kB/s eta 0:05:14
   ---------------------------------------- 0.8/203.1 MB 657.8 kB/s eta 0:05:08
   ---------------------------------------- 0.8/203.1 MB 657.8 kB/s eta 0:05:08
   ---------------------------------------- 1.0/203.1 MB 592.2 kB/s eta 0:05:42
   ---------------------------------------- 1.0/203.1 MB 592.2 kB/s eta 0:05:42
   ---------------------------------------- 1.0/203.1 MB 592.2 kB/s eta 0:05:42
   ---------------------------------------- 1.3/203.1 MB 588.8 kB/s eta 0:05:43
   ---------------------------------------- 1.3/203.1 MB 588.8 kB/s eta 0:05

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pennylane-qiskit 0.38.1 requires sympy<1.13, but you have sympy 1.13.1 which is incompatible.

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import pennylane as qml
from pennylane import numpy as np
import torch
from torch import nn
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
import pandas as pd

In [3]:
# Load the dataset
file_path = 'C:\\Users\\Nirusan03\\PycharmProjects\\FYP_POC\\Final_Dataset.csv'  # Update path
dataset = pd.read_csv(file_path)

# Selecting features and label
features = dataset.drop(columns=['Label', 'Label.1'])[:100]  # Use only a subset to reduce computation
label = dataset['Label'][:100]

# Encode and scale the data
label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(label)

scaler = StandardScaler()
features_scaled = scaler.fit_transform(features)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(features_scaled, encoded_labels, test_size=0.2, random_state=42)

In [4]:
# Define a 2-qubit quantum device
n_qubits = 2
dev = qml.device("default.qubit", wires=n_qubits)

# Quantum circuit to process input data
def quantum_circuit(inputs, weights):
    qml.templates.AngleEmbedding(inputs, wires=range(n_qubits))
    qml.templates.BasicEntanglerLayers(weights, wires=range(n_qubits))
    return [qml.expval(qml.PauliZ(i)) for i in range(n_qubits)]

In [5]:
# Quantum node
n_layers = 2
weight_shapes = {"weights": (n_layers, n_qubits)}
qnode = qml.QNode(quantum_circuit, dev, interface="torch")

# Define the hybrid quantum-classical layer
class HybridModel(nn.Module):
    def __init__(self):
        super(HybridModel, self).__init__()
        self.q_params = nn.Parameter(torch.randn(weight_shapes["weights"], dtype=torch.float32))  # Ensure float32
        self.fc1 = nn.Linear(features_scaled.shape[1], n_qubits)
        
        # Adjust fc2's input size based on actual output from qnode
        self.fc2 = nn.Linear(n_qubits, 1)  # Adjust if needed after confirming q_out size

    def forward(self, x):
        x = torch.tanh(self.fc1(x))
        
        # Execute the quantum node
        q_out = qnode(x, self.q_params)
        
        # Convert q_out to a tensor and stack if it is a list
        if isinstance(q_out, list):
            q_out = torch.stack([torch.tensor(val, dtype=torch.float32) for val in q_out])
        
        # Ensure q_out shape compatibility
        q_out = q_out.view(-1, n_qubits)  # Reshape q_out to have compatible shape for fc2
        
        # Pass through the final fully connected layer
        x = self.fc2(q_out)
        return torch.sigmoid(x)  # Output probability for binary classification


In [6]:
# Initialize the model, loss, and optimizer
model = HybridModel()
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

In [7]:
# Data Preparation Section

# Check that target values are in the range [0, 1]
unique_labels = set(y_train)
print("Unique values in y_train:", unique_labels)

# Convert to PyTorch tensors with correct data types
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
# Convert y_train to binary by mapping all values > 1 to 1
y_train = [0 if label == 0 else 1 for label in y_train]
y_train_tensor = torch.tensor(y_train, dtype=torch.float32).view(-1, 1)
print("Updated unique values in y_train:", set(y_train))

# Confirm target tensor is in the correct format for BCELoss
print("Target tensor dtype and shape:", y_train_tensor.dtype, y_train_tensor.shape)

Unique values in y_train: {0, 1, 2, 3}
Updated unique values in y_train: {0, 1}
Target tensor dtype and shape: torch.float32 torch.Size([80, 1])


In [8]:
# Test model with a single sample to check for compatibility
optimizer.zero_grad()
output = model(X_train_tensor[:1])  # Forward pass with a single sample
sample_loss = criterion(output, y_train_tensor[:1])  # Calculate sample loss
print("Sample loss calculation successful:", sample_loss.item())

Sample loss calculation successful: 0.7964736819267273


C:\Users\Nirusan03\AppData\Local\Temp\ipykernel_11116\1714999047.py:24: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  q_out = torch.stack([torch.tensor(val, dtype=torch.float32) for val in q_out])


In [19]:
n_epochs = 10
for epoch in range(n_epochs):
    optimizer.zero_grad()
    output = model(X_train_tensor)
    loss = criterion(output, y_train_tensor)
    loss.backward()
    optimizer.step()
    print(f"Epoch {epoch+1}, Loss: {loss.item()}")

Epoch 1, Loss: 0.639438271522522
Epoch 2, Loss: 0.6394311785697937
Epoch 3, Loss: 0.6394241452217102
Epoch 4, Loss: 0.6394169926643372
Epoch 5, Loss: 0.6394098401069641
Epoch 6, Loss: 0.6394027471542358
Epoch 7, Loss: 0.639395534992218
Epoch 8, Loss: 0.6393882632255554
Epoch 9, Loss: 0.6393809914588928
Epoch 10, Loss: 0.6393737196922302


C:\Users\Nirusan03\AppData\Local\Temp\ipykernel_11116\1714999047.py:24: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  q_out = torch.stack([torch.tensor(val, dtype=torch.float32) for val in q_out])
C:\Users\Nirusan03\AppData\Local\Temp\ipykernel_11116\1714999047.py:24: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  q_out = torch.stack([torch.tensor(val, dtype=torch.float32) for val in q_out])
C:\Users\Nirusan03\AppData\Local\Temp\ipykernel_11116\1714999047.py:24: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  q_out = torch.stack([torch.tenso

In [27]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

model.eval()
with torch.no_grad():
    # Define X_test_tensor and y_test_tensor
    X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
    y_test_tensor = torch.tensor(y_test, dtype=torch.float32).view(-1, 1)

    # Get the model's predictions
    y_pred_probs = model(X_test_tensor)
    y_pred = (y_pred_probs >= 0.5).float()  # Convert probabilities to binary labels

    # Convert tensors to numpy arrays for compatibility
    y_pred_np = y_pred.cpu().numpy()
    y_test_np = y_test_tensor.cpu().numpy()

    # Calculate evaluation metrics for binary classification
    accuracy = accuracy_score(y_test_np, y_pred_np)
    precision = precision_score(y_test_np, y_pred_np, average='macro', zero_division=1)
    recall = recall_score(y_test_np, y_pred_np, average='macro', zero_division=1)
    f1 = f1_score(y_test_np, y_pred_np, average='macro', zero_division=1)

    # Print evaluation results
    print(f"Accuracy: {accuracy:.4f}")
    print(f"Precision (macro): {precision:.4f}")
    print(f"Recall (macro): {recall:.4f}")
    print(f"F1 Score (macro): {f1:.4f}")

Accuracy: 0.4000
Precision (macro): 0.8500
Recall (macro): 0.2500
F1 Score (macro): 0.1429


C:\Users\Nirusan03\AppData\Local\Temp\ipykernel_11116\1714999047.py:24: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  q_out = torch.stack([torch.tensor(val, dtype=torch.float32) for val in q_out])
